In [1]:
from tqdm import tqdm

# Using the Torch Transformer Module

In [2]:
import torch
from torch.nn import Transformer, Embedding, LogSoftmax, NLLLoss
from torch.optim import SGD

In [4]:
n, k, V = 10, 3, 5
end = V

vecs = torch.randint(V-1, size=(n, k))+1
trgt = vecs.sort(-1)[0]
V += 1


trgt_ = torch.cat((trgt[:, 1:], torch.ones((n, 1))*end), dim=-1).long()

#vecs_ = vecs.transpose(0,1)
#trgt_ = trgt.transpose(0,1)

In [5]:
trgt, trgt_

(tensor([[2, 3, 4],
         [1, 3, 4],
         [1, 1, 2],
         [1, 2, 3],
         [2, 4, 4],
         [1, 2, 3],
         [1, 2, 4],
         [1, 1, 4],
         [1, 3, 3],
         [1, 3, 4]]),
 tensor([[3, 4, 5],
         [3, 4, 5],
         [1, 2, 5],
         [2, 3, 5],
         [4, 4, 5],
         [2, 3, 5],
         [2, 4, 5],
         [1, 4, 5],
         [3, 3, 5],
         [3, 4, 5]]))

In [16]:
class VtoP(torch.nn.Module):
    def __init__(self, vocab_dim):
        super().__init__()
        self.emb = Embedding(V, V)
        self.tr = Transformer(d_model=V, nhead=V//2)
        self.sigma = LogSoftmax(dim=-1)
        
    def forward(self, x, y):
        x_emb = self.emb(x)
        y_emb = self.emb(x)
        out = self.tr(x_emb, y_emb)
        out_p = self.sigma(out)
        return out_p

In [17]:
model = VtoP(V)
criterion = NLLLoss()
optimiser = SGD(model.parameters(), lr=0.01)

model.train()

losses = []

In [18]:
for _ in tqdm(range(1103)):
        optimiser.zero_grad()
        
        out_ = model(vecs, trgt)
        out = out_ # .argmax(-1)  # out_.transpose(0, 1).argmax(-1)
        
        loss = criterion(out.flatten(0,1), trgt.flatten(0,1))
            
        loss.backward()
        optimiser.step()
        
        losses.append(loss.detach().item())

  0%|          | 0/1103 [00:00<?, ?it/s]/home/valentin/.local/lib/python3.6/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag
100%|██████████| 1103/1103 [00:35<00:00, 30.69it/s]


In [ ]:
out.flatten(0,1).shape, trgt.flatten(0,1).shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(losses)), losses, "--")

In [ ]:
test_vecs_ = torch.randint(V, size=(10, k))
none_vecs_ = vecs.sort(-1)[0] #torch.zeros((10, k))


model.eval()


test_out = model(test_vecs_, none_vecs_)


print(test_vecs_)
print()
print(test_out.argmax(-1))

In [ ]:
model.eval()


test_out = model(test_vecs_, none_vecs_)


print(test_vecs_.transpose(0,1))
print()
print(test_out.transpose(0,1).argmax(-1))

## Using a TransformerDecoder Only

In [7]:
from torch.nn import TransformerDecoder, TransformerDecoderLayer

In [8]:
test_vecs_ = torch.randint(V, size=(10, k)).transpose(0,1)
none_vecs_ = torch.zeros((10, k)).transpose(0,1)


model.eval()


test_out = model(test_vecs_, none_vecs_)

print(test_out.argmax(-1))

NameError: name 'model' is not defined

---
# Using Huggingface's transformers library (BERT, GPT-2, ...)

In [ ]:
import torch
from transformers import BertConfig, BertModel

In [ ]:
conf = BertConfig()

model = BertModel(conf)

In [ ]:
tt = torch.tensor([[1,2,3]])
out = model(tt, return_dict=True)

In [ ]:
out.last_hidden_state.shape

In [ ]:
tt = torch.tensor([[1,2,3,4], [5,6,7,8]]).reshape(-1)